In [2]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 67 Dec 14 16:43 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 4.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=96d782659460389b66cf804f37d35de780c54d5827a6bae107ab8759e2ded359
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [3]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v


Kaggle API 1.5.6


In [4]:

! kaggle competitions download -c 2019-ml-finalproject

import zipfile
import os
os.mkdir('/content/input')

 75% 49.0M/65.0M [00:00<00:00, 30.1MB/s]
100% 65.0M/65.0M [00:00<00:00, 86.3MB/s]


In [0]:
zip_ref = zipfile.ZipFile("/content/2019-ml-finalproject.zip", 'r')
zip_ref.extractall("/content/input")
zip_ref.close()

In [6]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-4.1.2.30
     |████████████████████████████████| 25.0MB 94kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 76kB/s 


###############################################################################################################

In [0]:

import pandas as pd
import numpy as np
import cv2
from sklearn.cluster import KMeans
import copy
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import GridSearchCV

In [8]:
!pip install kmc2

     |████████████████████████████████| 102kB 5.5MB/s 
     |████████████████████████████████| 163kB 33.3MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252232 sha256=1a36709e0982531eedcdf1f4a4bf261ca87d07731be807ca48b2850e691b7a1f
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [9]:
labelll=pd.read_csv('../content/input/Label2Names.csv')
labelll.loc[100] = ['1', 'Faces']
labelll.loc[101] = ['102', 'BACKGROUND_Google']

print(labelll.iloc[0][1])
print(labelll)

Faces_easy
       1              Faces
0      2         Faces_easy
1      3           Leopards
2      4         Motorbikes
3      5          accordion
4      6          airplanes
..   ...                ...
97    99      windsor_chair
98   100             wrench
99   101           yin_yang
100    1              Faces
101  102  BACKGROUND_Google

[102 rows x 2 columns]


In [10]:
#데이터 로드


data1 = ('../content/input/train')
data2 = ('../content/input/testAll_v2')

train_data = list()
train_name = list()
test_data = list()

namefile = os.listdir(data1)
testimgfile = os.listdir(data2)
testimgfile.sort()

#train
for file in namefile:
  imgfile = os.listdir(data1 + '/' + file)
  
  #(라벨링)
  for i in range (0,102):
    if (file == labelll.iloc[i][1]):
      label = labelll.iloc[i][0] #백그라운드 제외

  for img in imgfile:
    if (label == '102'):
      print("OK")
      break
    image = cv2.imread(data1 + '/' + file + '/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (300,250), interpolation = cv2.INTER_LINEAR)
    train_data.append(image)
    train_name.append(label)

train_data = np.array(train_data)
train_name = np.array(train_name)


#test
for img in testimgfile:
    image = cv2.imread(data2 + '/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (300,250), interpolation = cv2.INTER_LINEAR)
    test_data.append(image)

test_data = np.array(test_data)


print(train_data.shape)  #( , 100, 200)
print(train_name)




OK
(3030, 250, 300)
['27' '27' '27' ... '10' '10' '10']


In [0]:
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 10
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        dense_feat = sift.compute(img, kp)
        x.append(dense_feat[1])
        
    return x

# extract dense sift features from training images
x_train = computeSIFT(train_data)
x_test = computeSIFT(test_data)


all_train_desc = []
for i in range(len(x_train)):
    for j in range(x_train[i].shape[0]):
        all_train_desc.append(x_train[i][j,:])

all_train_desc = np.array(all_train_desc)

In [0]:
import math

def extract_denseSIFT(img):
    DSIFT_STEP_SIZE = 2
    sift = cv2.xfeatures2d.SIFT_create()
    disft_step_size = DSIFT_STEP_SIZE
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img.shape[0], disft_step_size)
                for x in range(0, img.shape[1], disft_step_size)]

    descriptors = sift.compute(img, keypoints)[1]
    
    #keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors


# form histogram with Spatial Pyramid Matching upto level L with codebook kmeans and k codewords
def getImageFeaturesSPM(L, img, kmeans, k):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):                
                desc = extract_denseSIFT(img[y:y+h_step, x:x+w_step])                
                #print("type:",desc is None, "x:",x,"y:",y, "desc_size:",desc is None)
                predict = kmeans.predict(desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    # normalize hist
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist


# get histogram representation for training/testing data
def getHistogramSPM(L, data, kmeans, k):    
    x = []
    for i in range(len(data)):        
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

In [0]:
def clusterFeatures(all_train_desc, k):
    seeding = kmc2.kmc2(np.array(all_train_desc).reshape(-1,128), k)
    kmeans = MiniBatchKMeans(k, init=seeding).fit(np.array(all_train_desc).reshape(-1,128))
    return kmeans

In [15]:
import kmc2

k = 400
kmeans = clusterFeatures(all_train_desc, k)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)


In [0]:
train_histo = getHistogramSPM(2, train_data, kmeans, k) 
test_histo = getHistogramSPM(2, test_data, kmeans, k)

In [23]:
print(train_train_x.shape)

(2272, 8400)


In [0]:
clf = SVC(random_state=0, C=0.0298)
clf.fit(train_histo,train_name)
predict = clf.predict(test_histo)

############################################################################################

In [18]:
# numpy 를 Pandas 이용하여 결과 파일로 저장


import pandas as pd

d = {'Id': testimgfile, 'Category': predict}
df = pd.DataFrame(data=d)

print(df)

                  Id Category
0     image_0001.jpg       77
1     image_0002.jpg       14
2     image_0003.jpg       26
3     image_0004.jpg       51
4     image_0005.jpg       76
...              ...      ...
1687  image_1688.jpg       26
1688  image_1689.jpg       61
1689  image_1690.jpg       29
1690  image_1691.jpg       38
1691  image_1692.jpg       34

[1692 rows x 2 columns]


In [0]:
df.to_csv('sohyeon_project.csv',index=False, header=True)

In [20]:
! kaggle competitions submit -c 2019-ml-finalproject -f sohyeon_project.csv -m "BaekSohyeon"

100% 29.6k/29.6k [00:03<00:00, 8.76kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 

22.163


*   spm함
*   빠르게
* 스텝15

